In [1]:
import pandas as pd

In [19]:
dfs = []
dfs.append(pd.read_csv('Departures hist data/Detailed_Statistics_Departures.csv'))
for i in range(2, 25):
    path = 'Departures hist data/Detailed_Statistics_Departures-' + str(i) + '.csv'
    dfs.append(pd.read_csv(path))

In [24]:
all_departures = pd.concat(dfs)

In [25]:
len(all_departures)

3272931

In [26]:
all_departures = all_departures.drop_duplicates()

In [27]:
len(all_departures)

2653114

In [29]:
all_departures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653114 entries, 0 to 684707
Data columns (total 17 columns):
 #   Column                                    Dtype 
---  ------                                    ----- 
 0   Carrier Code                              object
 1   Date (MM/DD/YYYY)                         object
 2   Flight Number                             int64 
 3   Tail Number                               object
 4   Destination Airport                       object
 5   Scheduled departure time                  object
 6   Actual departure time                     object
 7   Scheduled elapsed time (Minutes)          int64 
 8   Actual elapsed time (Minutes)             int64 
 9   Departure delay (Minutes)                 int64 
 10  Wheels-off time                           object
 11  Taxi-Out time (Minutes)                   int64 
 12  Delay Carrier (Minutes)                   int64 
 13  Delay Weather (Minutes)                   int64 
 14  Delay National Avia

In [30]:
all_departures = all_departures[['Carrier Code', 
                                 'Date (MM/DD/YYYY)', 
                                 'Flight Number', 
                                 'Destination Airport', 
                                 'Scheduled departure time', 
                                 'Departure delay (Minutes)']]

In [32]:
all_departures.reset_index()

,index,Carrier Code,Date (MM/DD/YYYY),Flight Number,Destination Airport,Scheduled departure time,Departure delay (Minutes)
0,0,VX,01/01/2012,201,SFO,07:00,3
1,1,VX,01/01/2012,233,LAX,15:50,12
2,2,VX,01/01/2012,811,LAX,08:40,0
3,3,VX,01/01/2012,815,LAX,18:30,27
4,4,VX,01/01/2012,837,SFO,14:05,-2
...,...,...,...,...,...,...,...
2653109,684703,UA,12/31/2021,2669,EWR,15:45,0
2653110,684704,UA,12/31/2021,2677,IAD,13:00,-5
2653111,684705,UA,12/31/2021,2678,IAH,12:45,10
2653112,684706,UA,12/31/2021,2682,IAH,14:05,1


In [33]:
all_departures['Date'] = pd.to_datetime(all_departures['Date (MM/DD/YYYY)'])

In [35]:
all_departures = all_departures.drop(columns = ['Date (MM/DD/YYYY)'])

In [37]:
all_departures = all_departures.rename(columns={"Flight Number":"Flight",
                                                "Destination Airport" : "Destination airport",
                                                "Scheduled departure time" : "Scheduled time",
                                                 "Departure delay (Minutes)" : "Delay"})

In [38]:
all_departures

,Carrier Code,Flight,Destination airport,Scheduled time,Delay,Date
0,VX,201,SFO,07:00,3,2012-01-01
1,VX,233,LAX,15:50,12,2012-01-01
2,VX,811,LAX,08:40,0,2012-01-01
3,VX,815,LAX,18:30,27,2012-01-01
4,VX,837,SFO,14:05,-2,2012-01-01
...,...,...,...,...,...,...
684703,UA,2669,EWR,15:45,0,2021-12-31
684704,UA,2677,IAD,13:00,-5,2021-12-31
684705,UA,2678,IAH,12:45,10,2021-12-31
684706,UA,2682,IAH,14:05,1,2021-12-31


In [39]:
all_departures.to_csv("Delays_historical.csv")